In [1]:
#import 
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics.pairwise import euclidean_distances
import numpy as np
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import f1_score
from sklearn.metrics import cohen_kappa_score
from sklearn.neighbors import KNeighborsClassifier
from sklearn import svm
from sklearn.feature_selection import RFE

In [2]:
#parameter
k=2
m=2
N=2 #number of class  
tou =0.3 # threshold

#fuzzyfier
fuzzyfier = 2/(m-1) 

In [3]:
#reading the microarray gene expression dataset
lu = pd.read_csv('prostatemattest.txt', header=None,sep='\t')

#lu.isnull().values.any()
lu.head(3)

,0,1,2,3,4,5,6,7,8,9,...,12524,12525,12526,12527,12528,12529,12530,12531,12532,12533
0,-11.4,2.7,0.6,4.3,28,0.3,-17.5,-5.4,7.5,-0.9,...,4.1,2.5,5,22.5,7.3,14,19.3,-39.5,37.3,1
1,-1.0,-1.0,0.0,-1.0,3,0.0,-3.0,1.0,-2.0,0.0,...,10.0,2.0,4,5.0,1.0,6,6.0,0.0,26.0,1
2,-9.0,-19.0,0.0,0.0,76,9.0,-57.0,35.0,23.0,3.0,...,-66.0,12.0,14,59.0,16.0,-13,-10.0,-156.0,-21.0,1


In [4]:
#dividing the data into train-test and labeled- u dataset to 
y = lu.iloc[:,[-1]]
X = lu.iloc[:,:-2]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.85, random_state=100)



In [5]:
y_train.value_counts()

12533
2        8
1        7
dtype: int64

In [6]:
###### RFE
from sklearn.feature_selection import RFE
from sklearn.linear_model import LinearRegression




lm = LinearRegression()
lm.fit(X_train,y_train)


rfe = RFE(lm, n_features_to_select=150)
rfe = rfe.fit(X_train, y_train)
X_train_rfe = X_train[X_train.columns[rfe.support_]]
X_test_rfe = X_test[X_test.columns[rfe.support_]]


In [7]:
'''X_train_rfe =X_train
X_test_rfe = X_test'''



'X_train_rfe =X_train\nX_test_rfe = X_test'

In [8]:
X_Labeled , X_unLabeled, y_Labeled, y_unLabeled = train_test_split(X_train_rfe, y_train, test_size=0.60, random_state=80)

In [9]:
y_Labeled.value_counts()

12533
2        3
1        3
dtype: int64

In [10]:
#computing euclidean distance for each unlabeled data point to labeled data point
distul = euclidean_distances(X_unLabeled, X_Labeled)

In [11]:
#array of index of nearest neighbour
knn_index=[]
for i in range(len(distul)) :
    knn_index.append (np.argsort(distul[i]))

In [12]:
#function to get distance from knn and its class

def para_knn(i):
    #print (distul[i])
    #print (knn_index[i][:k])
    ind = knn_index[i][:k]
    #print([i for i in distul[i][ind]])
    k_dist = [i for i in distul[i][ind]]
    k_dist_fuzzifier =  np.power(k_dist,fuzzyfier) 
    k_dist = sum(k_dist_fuzzifier)
    #print(y_Labeled.iloc[ind])
       
    
    return k_dist,y_Labeled.iloc[ind].values

In [13]:
#relationship R

def R(ui,lx):
    numerator = para_knn(ui)[0]
    denomerator = distul[ui][lx]
    denomerator = np.power(denomerator,fuzzyfier)
    return (numerator/denomerator)

    

In [14]:
#class function 

def C(j,lx):
    if (y_Labeled.iloc[lx].values== j):
        return 1
    else:
        return 0

In [15]:

#lower approximation function

def R_lower(ui,cj):
    rl=[]
    for i in range(k):
        l_x = knn_index[ui][i]
        rl.append(min(1,(1 + C(cj,l_x)) -R(ui,l_x)) )
    return min(rl)    
            
        

In [16]:
#upper approximation function

def R_upper(ui,cj):
    ru=[]
    for i in range(k):
        l_x = knn_index[ui][i]
        
        ru.append( max(0,(R(ui,l_x)+C(cj,l_x))-1) )
    return max(ru) 

In [17]:
#get the most confusing sample from    
def get_R():
    
    #computing the lower approximation for unlabeled with association of class 1
    R_lower_1=[]
    for i in range(len(X_unLabeled)):
        R_lower_1.append(R_lower(i,1))
        
    #computing the lower approximation for unlabeled with association of class 2    
    R_lower_2=[]
    for i in range(len(X_unLabeled)):
        R_lower_2.append(R_lower(i,2)) 
     
    #computing the upper approximation for unlabeled with association of class 1
    R_upper_1=[]
    for i in range(len(X_unLabeled)):
        R_upper_1.append(R_upper(i,1))
    
    #computing the upper approximation for unlabeled with association of class 2
    R_upper_2=[]
    for i in range(len(X_unLabeled)):
        R_upper_2.append(R_upper(i,2))    

    
    #getting average of the upper and lower approximation and computing normalized ratio for the class 1
    avg_for_1= np. add(R_lower_1, R_upper_1)
    avg_for_1 =[e/2 for e in avg_for_1]
    max_h1=max(avg_for_1)
    avg_for_1 = [e/max_h1 for e in avg_for_1]
    
    #getting average of the upper and lower approximation and computing normalized ratio for the class 1 
    avg_for_2= np. add(R_lower_2, R_upper_2)
    avg_for_2 =[e/2 for e in avg_for_2]
    max_h2=max(avg_for_2)
    avg_for_2 = [e/max_h2 for e in avg_for_2]
    max_ratio=0
    conf_index=0
    
    for i in range(len(y_unLabeled)):
        if (y_unLabeled.iloc[i].values == 2):
            print('ind: ',[i],avg_for_1[i])
            if(avg_for_1[i]>max_ratio):
                max_ratio =avg_for_1[i]
                conf_index = X_unLabeled.index[i]
                
   
    for i in range(len(y_unLabeled)):
        if (y_unLabeled.iloc[i].values == 1):
            print('ind: ',[i],avg_for_2[i])
            if(avg_for_2[i]>max_ratio):
                max_ratio =avg_for_2[i]
                conf_index = X_unLabeled.index[i]
                
                
    if (max_ratio >= tou):
        return ( max_ratio, conf_index)
    return ( max_ratio, conf_index)

In [18]:
#function for transfer of sample for unlabeled data pattern to labeled data pattern
def transfer_sample(In):
    global X_Labeled 
    global y_Labeled 
    global X_unLabeled 
    global y_unLabeled 
    X_Labeled = X_Labeled.append(X_unLabeled.loc[In])
    y_Labeled = y_Labeled.append(y_unLabeled.loc[In])
    X_unLabeled = X_unLabeled.drop([In])
    y_unLabeled = y_unLabeled.drop([In])
    print('sampled moved')


In [19]:

while(get_R()[0]>tou):
    transfer_sample(get_R()[1])


ind:  [0] 0.0
ind:  [1] 0.0
ind:  [4] 0.9018719754592065
ind:  [5] 0.8188964025473224
ind:  [7] 0.4981814675197995
ind:  [2] 0.8522517267769305
ind:  [3] 0.7457006102617011
ind:  [6] 0.0
ind:  [8] 0.0
ind:  [0] 0.0
ind:  [1] 0.0
ind:  [4] 0.9018719754592065
ind:  [5] 0.8188964025473224
ind:  [7] 0.4981814675197995
ind:  [2] 0.8522517267769305
ind:  [3] 0.7457006102617011
ind:  [6] 0.0
ind:  [8] 0.0
sampled moved
ind:  [0] 0.0
ind:  [1] 0.0
ind:  [4] 0.9018719754592065
ind:  [6] 1.0
ind:  [2] 0.8522517267769305
ind:  [3] 0.7457006102617011
ind:  [5] 0.18110359745267757
ind:  [7] 0.5018185324802005
ind:  [0] 0.0
ind:  [1] 0.0
ind:  [4] 0.9018719754592065
ind:  [6] 1.0
ind:  [2] 0.8522517267769305
ind:  [3] 0.7457006102617011
ind:  [5] 0.18110359745267757
ind:  [7] 0.5018185324802005
sampled moved
ind:  [0] 0.0
ind:  [1] 0.0
ind:  [4] 0.9018719754592065
ind:  [2] 0.8522517267769305
ind:  [3] 0.7457006102617011
ind:  [5] 0.18110359745267757
ind:  [6] 0.0
ind:  [0] 0.0
ind:  [1] 0.0
ind:  [

C:\anaconda\lib\site-packages\ipykernel_launcher.py:29: RuntimeWarning: invalid value encountered in double_scalars


In [20]:
len(X_Labeled)

13

In [21]:
#testing 

#computing euclidean distance for each test data point to labeled data point
disttl = euclidean_distances(X_test_rfe, X_Labeled)

#array of index of nearest neighbour
knn_index_test=[]
for i in range(len(disttl)) :
    knn_index_test.append (np.argsort(disttl[i]))
    
#function to get distance from knn and its class
def para_knn_test(i):
    ind = knn_index_test[i][:k]
    k_dist_test = [i for i in disttl[i][ind]]
    k_dist_fuzzifier_test =  np.power(k_dist_test,fuzzyfier) 
    k_dist_test = sum(k_dist_fuzzifier_test)
    return k_dist_test,y_Labeled.iloc[ind].values   

#relationship R for tests
def R(ti,lx):
    numerator = para_knn_test(ti)[0]
    denomerator = disttl[ti][lx]
    denomerator = np.power(denomerator,fuzzyfier)
    return (numerator/denomerator)

#class function test
def C(j,lx):
    if (y_Labeled.iloc[lx].values== j):
        return 1
    else:
        return 0
    
#lower approximation function for test

def R_lower_test(ti,cj):
    rl=[]
    for i in range(k):
        l_x = knn_index_test[ti][i]
        rl.append(min(1,(1 + C(cj,l_x)) -R(ti,l_x)) )
    return min(rl) 

#upper approximation function for test 

def R_upper_test(ti,cj):
    ru=[]
    for i in range(k):
        l_x = knn_index_test[ti][i]
        
        ru.append( max(0,(R(ti,l_x)+C(cj,l_x))-1) )
    return max(ru) 
            
    

In [22]:
#assigning class label
#get the most confusing sample from    
def get_Class():
    
    #computing the lower approximation for unlabeled with association of class 1
    R_lower_test_1=[]
    for i in range(len(X_test_rfe)):
        R_lower_test_1.append(R_lower_test(i,1))
        
    #computing the lower approximation for unlabeled with association of class 2    
    R_lower_test_2=[]
    for i in range(len(X_test_rfe)):
        R_lower_test_2.append(R_lower_test(i,2)) 
     
    #computing the upper approximation for unlabeled with association of class 1
    R_upper_test_1=[]
    for i in range(len(X_test_rfe)):
        R_upper_test_1.append(R_upper_test(i,1))
    
    #computing the upper approximation for unlabeled with association of class 2
    R_upper_test_2=[]
    for i in range(len(X_test_rfe)):
        R_upper_test_2.append(R_upper_test(i,2))    

    
    #getting average of the upper and lower approximation and computing normalized ratio for the class 1
    avg_test_1= np. add(R_lower_test_1, R_upper_test_1)
    avg_test_1 =[e/2 for e in avg_test_1]
    
   
    
    #getting average of the upper and lower approximation and computing normalized ratio for the class 1 
    avg_test_2= np. add(R_lower_test_2, R_upper_test_2)
    avg_test_2 =[e/2 for e in avg_test_2]
    
    predicted_label = [0] * len(avg_test_1)
    for i in range(len(avg_test_1)):
        if (avg_test_1[i]>=avg_test_2[i]):
            predicted_label[i]=1
        else:
            predicted_label[i]=2
            
   
    return predicted_label


In [23]:
predicted_label = get_Class()

In [24]:
#performance evaluation metrics

In [25]:
#performance evaluation metrics function
def get_per_mat(y_test,predicted_label):
    
    #Confusion matrix
    cf = confusion_matrix(y_test,predicted_label)
    print('confusion matrix: \n',cf)
    
    #accuracy
    print('accuracy: ',accuracy_score(y_test,predicted_label))
    
    #precision
    precision = cf[0][0]/(cf[0][0]+cf[1][0])
    print('precision: ',precision)
    
    #recall
    recall = cf[0][0]/(cf[0][0]+cf[0][1])
    print('recall: ',recall)
    
    #f-measure
    print('f-measure: ',f1_score(y_test,predicted_label))
    
    #cohen kappa score
    print('cohen kappa: ',cohen_kappa_score(y_test,predicted_label))
    
    return cf

In [26]:
get_per_mat(y_test,predicted_label)

confusion matrix: 
 [[31 12]
 [ 5 39]]
accuracy:  0.8045977011494253
precision:  0.8611111111111112
recall:  0.7209302325581395
f-measure:  0.7848101265822784
cohen kappa:  0.6084193804606831


array([[31, 12],
       [ 5, 39]], dtype=int64)

### KNN

In [27]:
print(y_Labeled.shape)
y_Labeled.value_counts()

(13, 1)


12533
1        7
2        6
dtype: int64

In [34]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.87, random_state=100)


In [35]:
y_train.shape

(13, 1)

In [40]:


model = KNeighborsClassifier(n_neighbors=2)
model.fit(X_train,y_train)

predicted= model.predict(X_test) 


C:\anaconda\lib\site-packages\ipykernel_launcher.py:2: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  


In [41]:
get_per_mat(y_test,predicted)

confusion matrix: 
 [[39  5]
 [27 18]]
accuracy:  0.6404494382022472
precision:  0.5909090909090909
recall:  0.8863636363636364
f-measure:  0.7090909090909092
cohen kappa:  0.28478151682571573


array([[39,  5],
       [27, 18]], dtype=int64)

### SVM

In [32]:

classifier = svm.SVC(kernel='linear') # We set a SVM classifier, the default SVM Classifier (Kernel = Radial Basis Function)
classifier.fit(X_train, y_train) 
prediction_SVM = classifier.predict(X_test)

C:\anaconda\lib\site-packages\sklearn\utils\validation.py:72: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(**kwargs)


In [33]:
get_per_mat(y_test,prediction_SVM)

confusion matrix: 
 [[30 14]
 [ 6 39]]
accuracy:  0.7752808988764045
precision:  0.8333333333333334
recall:  0.6818181818181818
f-measure:  0.7499999999999999
cohen kappa:  0.5495951417004048


array([[30, 14],
       [ 6, 39]], dtype=int64)